In [ ]:
import json

In [ ]:
[x.delete() for x in Manuscript.objects.filter(lib_code=None)]

In [ ]:
items = MsImage.objects.filter(manuscript=None)
items.count()

In [ ]:
for x in items:
    try:
        ms_id = int(json.loads(x.orig_data_csv)["idmanuscripts"])
    except:
        x.delete()
        continue
    ms = Manuscript.objects.get(legacy_pk=ms_id)
    x.manuscript = ms
    x.save()
        

In [ ]:
items = MsPart.objects.filter(part_of_manuscript=None)
items.count()

In [ ]:
for x in items:
    try:
        ms_id = int(json.loads(x.orig_data_csv)["idmanuscripts"])
    except:
        x.delete()
        continue
    try:
        ms = Manuscript.objects.get(legacy_pk=ms_id)
    except:
        print(ms_id, x.id)
        continue
    x.part_of_manuscript = ms
    x.save()

In [1]:
# link lib to lit

In [2]:
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine
from django.conf import settings
from django.core.exceptions import ObjectDoesNotExist

In [3]:
dbc = settings.LEGACY_DB_CONNECTION
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [5]:
source_name = "litkat"
query = f"SELECT * FROM {source_name}"
df = pd.read_sql(query, con=db_connection)

In [6]:
df

,id,lit_id,libCode,sort,modified
0,1,6486,AT1200,1,0000-00-00 00:00:00
1,2,7923,AT3300,2,0000-00-00 00:00:00
2,3,2626,AT3500,,0000-00-00 00:00:00
3,4,6224,AT5400,,0000-00-00 00:00:00
4,5,2751,AT5400,,0000-00-00 00:00:00
...,...,...,...,...,...
322,456,10737,,,2019-12-16 10:03:02
323,457,10750,AT3800,,2020-01-08 11:55:06
324,458,8175,AT7290,,2020-05-22 10:21:40
325,459,11076,AT5200,,2020-10-12 20:10:19


In [22]:
for lib_code, ndf in df.groupby("libCode"):
    lib = Bibliothek.objects.filter(lib_code=lib_code).first()
    lit_ids = set(ndf["lit_id"].to_list())
    items = Literatur.objects.filter(legacy_pk__in=lit_ids)
    lib.literatur.add(*items)    

12
3
3
1
1
1
3
1
1
1
3
3
1
1
2
2
1
1
7
2
2
1
7
4
3
1
1
2
15
1
1
1
2
1
8
2
3
3
3
1
3
2
1
1
5
1
2
6
1
2
1
1
4
2
2
2
1
1
2
2
1
1
7
5
2
1
2
3
2
2
2
1
1
2
2
3
3
1
1
5
1
1
1
1
2
1
1
2
2
1
2
3
1
2
76
2
2
1
1
2
2
1
1
5
1
3
2
1
1


In [19]:
lit_ids

{9975}